In [1]:
!which python

/c/Users/elior/micromamba/envs/price_scraper_py311/python


First, some imports

In [2]:
import requests  # handles HTTP requests to fetch webpage content
from bs4 import BeautifulSoup  # parses and extracts data from HTML pages
import time  # to add delays between requests
import random  # to randomize user-agent selection
import json  # for saving output in JSON format
import logging  # to log events and errors for debugging
from datetime import datetime  # to timestamp scraped data

Here we define a list of real-like user-agent strings. Randomizing the user-agent makes requests more reliable because, without it, pythons requests module might send a generic identifier that websites can detect as a bot

In [3]:
# this will probably be stored in utils.py file
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/600.8.9 (KHTML, like Gecko) Version/8.0.8 Safari/600.8.9",
    "Mozilla/5.0 (iPad; CPU OS 8_4_1 like Mac OS X) AppleWebKit/600.1.4 (KHTML, like Gecko) Version/8.0 Mobile/12H321 Safari/600.1.4",
    "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.85 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.85 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_4) AppleWebKit/600.7.12 (KHTML, like Gecko) Version/8.0.7 Safari/600.7.12",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.10; rv:40.0) Gecko/20100101 Firefox/40.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/600.8.9 (KHTML, like Gecko) Version/7.1.8 Safari/537.85.17",
    "Mozilla/5.0 (iPad; CPU OS 8_4 like Mac OS X) AppleWebKit/600.1.4 (KHTML, like Gecko) Version/8.0 Mobile/12H143 Safari/600.1.4",
    "Mozilla/5.0 (Windows NT 6.1; rv:40.0) Gecko/20100101 Firefox/40.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3) AppleWebKit/600.6.3 (KHTML, like Gecko) Version/8.0.6 Safari/600.6.3",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:38.0) Gecko/20100101 Firefox/38.0",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.10; rv:38.0) Gecko/20100101 Firefox/38.0",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 8_4_1 like Mac OS X) AppleWebKit/600.1.4 (KHTML, like Gecko) CriOS/45.0.2454.68 Mobile/12H321 Safari/600.1.4",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.85 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.85 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/600.5.17 (KHTML, like Gecko) Version/7.1.5 Safari/537.85.14",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.85 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:40.0) Gecko/20100101 Firefox/40.0",
    "Mozilla/5.0 (Windows NT 6.3; WOW64; rv:39.0) Gecko/20100101 Firefox/39.0",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64; Trident/7.0; rv:11.0) like Gecko",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.11) AppleWebKit/601.1.56 (KHTML, like Gecko) Version/9.0 Safari/601.1.56",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; MDDRJS; rv:11.0) like Gecko",
    "Mozilla/5.0 (Windows NT 6.3; Trident/7.0; Touch; rv:11.0) like Gecko",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.155 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/600.3.18 (KHTML, like Gecko) Version/7.1.3 Safari/537.85.12",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko; Google Web Preview) Chrome/27.0.1453 Safari/537.36",
    "Mozilla/5.0 (iPad; CPU OS 8_0 like Mac OS X) AppleWebKit/600.1.4 (KHTML, like Gecko) Version/8.0 Mobile/12A365 Safari/600.1.4",
    "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.155 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; rv:39.0) Gecko/20100101 Firefox/39.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10) AppleWebKit/600.2.5 (KHTML, like Gecko) Version/8.0.2 Safari/600.2.5",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.85 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.71 Safari/537.36",
]

We're going to first create a base scraper, from which a more specific scraper (e.g. "The Rejected Shop") can extend this class for specific usage.
The class provides commont tools for web scraping. It sends HTTP requests with a random user-agent to help avoid detection. The class also handles errors and adds a delay between requests to mimic human browsing.

In [4]:
class BaseScraper:
    def __init__(self, base_url):
        #  stores the main URL of the website we're scraping, will be used in a subclass (specific website scraper)
        self.base_url = base_url
        
        # set up logging for debugging and tracking issues.
        self.setup_logging()
        
        self.request_http_headers = {
            'User-Agent': self.get_random_user_agent()
        }
    
    def get_random_user_agent(self):
        """
        returns a random user-agent string from a predefined list.
        rotating the User-Agent helps avoid anti-scraping blocks.
        """

        if not USER_AGENTS:
            logging.warning("USER_AGENTS list is empty, using a default User-Agent.")
            return "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:30.0) Gecko/20100101 Firefox/30.0"
        
        return random.choice(USER_AGENTS)
    
    def setup_logging(self):
        """
        configures the logging settings to output informational messages.
        info/warning/error/critical
        """
        logging.basicConfig(level=logging.INFO)
    
    def fetch_page(self, url):
        """
        makes an HTTP GET request to the specified URL using the set headers.        
        Parameters:
            url (str): The URL of the page to fetch.
        Returns:
            str: HTML content of the page if successful, None otherwise.
        """
        try:
            # send the HTTP request with the current headers
            # timeout=10 to prevent the request from hanging indefinitely if the website is slow 
            response = requests.get(url, headers=self.request_http_headers, timeout=10)
            
            # raise an error if the HTTP request returned an unsuccessful status code
            response.raise_for_status()
            
            # introduce a delay to mimic human browsing behavior and reduce the risk of being blocked
            sleep_time = random.uniform(1,3)
            logging.info(f"sleeping for {sleep_time:.2f} seconds before the next request.")
            time.sleep(sleep_time)
            
            # the HTML content
            return response.text
            
        except requests.RequestException as e:
            logging.error(f"Error fetching {url}: {e}")
            return None

    def parse_page(self, html):
        """
        This method should be implemented by subclasses.
        
        Parameters:
            html (str): The HTML content of a webpage.
        
        Raises:
            NotImplementedError: if the subclass does not override this method.
        """
        raise NotImplementedError("Subclasses must implement the parse_page method. Define how to extract data from HTML.")

let's test the BaseScraper class before we move to a website-specific scraper. We gonna try with a valid url and then a url that is not valid.

In [5]:
# init a base scraper
scraper = BaseScraper("http://example.com")

# test with a valid URL
html_content = scraper.fetch_page("http://example.com")
if html_content:
    print("Fetched page successfully!")
else:
    print("Failed to fetch page.")

# test error handling with an invalid URL
invalid_content = scraper.fetch_page("http://thisurldoesnotexist1234.com")
if invalid_content is None:
    print("Error handling works for invalid URL.")
else:
    print("Unexpected success for invalid URL.")

INFO:root:sleeping for 1.99 seconds before the next request.
ERROR:root:Error fetching http://thisurldoesnotexist1234.com: HTTPConnectionPool(host='thisurldoesnotexist1234.com', port=80): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000002561A80B550>: Failed to resolve 'thisurldoesnotexist1234.com' ([Errno 11001] getaddrinfo failed)"))


Fetched page successfully!
Error handling works for invalid URL.


Looks like everything is working as it should be. For the invalid url the error tells that the program tried to access a non-existent website and failed to resolve its address after multiple attempts.

Now let's create a more specific scraper, one for the "The Reject Shop". Before we do it though, for now we will create a mapping that maps sku's to URLs, this can be changed to dynamic search later if decided.

In [7]:
SKU_URL_MAPPING = {
    "30061292": "https://www.rejectshop.com.au/p/palmolive-naturals-shampoo-coconut-cream-350ml",
    "30113527": "https://www.rejectshop.com.au/p/whiskas-jellymeat-400g",
    "30115549": "https://www.rejectshop.com.au/p/twisties-party-bag-cheese-270g",
    "30043588": "https://www.rejectshop.com.au/p/quilton-aloe-vera-tissue-3ply-95pk",
    "30087959": "https://www.rejectshop.com.au/p/jif-surface-cleaner-lemon-scent-500ml",
}

In [17]:
class RejectShopScraper(BaseScraper):
    def __init__(self, base_url):
        # initialize the BaseScraper with the main website URL
        # Note: currently the base_url instance variable is not used here as well.
        # The reason is we're using a predefined list of URL's gathered manually, thus, if we will decide
        # to change this approach and go for a more automatically one, this base_url placeholder will be useful.
        super().__init__(base_url)
    
    def construct_product_url(self, sku):
        """
        constructs the full URL for a product given its SKU.
        
        Note: currently we look up the SKU in a predefined mapping.
        this implementation can be replaced with a dynamic search mechanism to retrieve the product URL.
        the method returns the product URL regardless of how it's obtained, keeping the interface consistent.
        """
        try:
            return SKU_URL_MAPPING[sku]
        except KeyError:
            logging.error(f"SKU {sku} not found in the URL mapping.")
            return None
    
    def parse_page(self, html, sku):
        """
        parses the HTML content of a product page to extract details.
        Parameters:
            html (str): the HTML content of the product page.
            sku (str): the SKU parameter provided for reference.
        Returns:
            dict: a dictionary containing the SKU, product name, price, and current date.
        """
        soup = BeautifulSoup(html, 'html.parser')
        try:
            # Note: The selectors below (e.g., data-testid for product title, class names for SKU and price) 
            # were determined by manually inspecting the website's HTML structure.
            # It might be worth to check if a Machine Learning (NLP) approach would work to automate this (if there's a lot of websites to scrape)
            
            # extract product name using the data-testid attribute
            product_name_element = soup.find("h1", {"data-testid": "product-title"})
            print("DEBUG - Found product title element:", product_name_element)
            product_name = product_name_element.get_text(strip=True) if product_name_element else "Unknown Product"
            print("DEBUG - Extracted product name:", product_name)
            
            # extract the SKU from the <p> tag with class "pdp-sku except-phone"
            sku_element = soup.find("p", class_="jsx-ac1f85233799a587 pdp-sku except-phone")
            print("DEBUG - Found product SKU element:", sku_element)
            
            if sku_element:
                sku_text = sku_element.get_text(strip=True)
                # Remove the "SKU:" prefix and extra spaces
                extracted_sku = sku_text.replace("SKU:", "").strip()
                print("DEBUG - Extracted SKU:", extracted_sku)
            else:
                extracted_sku = "SKU Not Found"
            
            # extract the product price from the <div> with class "product-price"
            price_container = soup.find("div", class_="jsx-c5b8eb4ab4d5ad55 product-price")
            print("DEBUG - Price Container:", price_container)
            if price_container:
                # combine the text from all spans (e.g., "$", "3", ".45") and remove extra spaces
                raw_price_text = price_container.get_text(strip=True)
                product_price = raw_price_text
                print("DEBUG - Product Price:", product_price)
            else:
                product_price = "Price Not Found"
            
            # get the current date in YYYY-MM-DD format
            current_date = datetime.now().strftime("%Y-%m-%d")
            
            return {
                "SKU": extracted_sku,
                "Product Name": product_name,
                "Price": product_price,
                "Date": current_date
            }
        except Exception as e:
            logging.error(f"Error parsing HTML for SKU {sku}: {e}")
            return None
    
    def scrape_product_by_sku(self, sku):
        """
        scrapes the product page for a given SKU and returns the extracted data.
        Parameters:
            sku (str): The product SKU to scrape.
        Returns:
            dict: The extracted product details or None if scraping fails.
        """
        product_url = self.construct_product_url(sku)
        logging.info(f"Scraping product URL: {product_url}")
        
        # fetch the page content using the BaseScraper fetch_page method
        html_content = self.fetch_page(product_url)
        print("DEBUG", "\n")
        print(html_content)
        if html_content:
            product_data = self.parse_page(html_content, sku)
            if product_data:
                logging.info(f"Successfully scraped data for SKU {sku}")
            else:
                logging.error(f"Failed to parse product data for SKU {sku}")
            return product_data
        else:
            logging.error(f"Failed to fetch page for SKU {sku}")
            return None

let's test this class and verify it works as it should be

In [18]:
base_url = "https://www.rejectshop.com.au"

reject_shop_scraper = RejectShopScraper(base_url)

print(f"testing SKU:{30087959}")
product_data = reject_shop_scraper.scrape_product_by_sku(sku)
print(product_data, "\n")

# list of test SKUs, used as keys in SKU_URL_MAPPING
# test_skus = ["30061292", "30113527", "30115549", "30043588", "30087959"]

# loop through each sku, scrape the product page, and print the results
# for sku in test_skus:
#     print(f"testing SKU: {sku}")
#     product_data = reject_shop_scraper.scrape_product_by_sku(sku)
#     print(product_data, "\n")

INFO:root:Scraping product URL: https://www.rejectshop.com.au/p/jif-surface-cleaner-lemon-scent-500ml


testing SKU:30087959


INFO:root:sleeping for 2.00 seconds before the next request.
INFO:root:Successfully scraped data for SKU 30087959


DEBUG 

<!DOCTYPE html><html data-theme="light"><head><meta charSet="UTF-8"/><meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0,user-scalable=0"/><meta name="apple-mobile-web-app-capable" content="yes"/><meta name="apple-mobile-web-app-status-bar-style" content="black-translucent"/><meta name="robots" content="index,follow"/><meta name="keywords" content="essentials, discount, grocery, saving, cleaning"/><meta name="twitter:card" content="summary_large_image"/><meta property="og:type" content="website"/><link rel="icon" href="/images/favicon-icon-16x16.jpeg"/><link rel="icon" href="/images/favicon-icon-32x32.jpeg"/><link rel="icon" href="/images/favicon-icon-64x64.jpeg"/><link rel="icon" href="/images/favicon-icon-128x128.jpeg"/><script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.